In [0]:
from pyspark.ml import PipelineModel
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col, when

# Carregar modelos e dados
slicer_model = PipelineModel.load("/FileStore/models/slicer_top10")
train_ready = spark.read.format("delta").load("/FileStore/data/train_ready")
val_ready = spark.read.format("delta").load("/FileStore/data/val_ready")

# Aplicar slicer
train_topk = slicer_model.transform(train_ready)
val_topk = slicer_model.transform(val_ready)

# Balanceamento leve
minority_df = train_topk.filter(col("label") == 1)
majority_df = train_topk.filter(col("label") != 1)
train_balanced = majority_df.sample(False, 0.1, seed=42).union(minority_df)

# Criar coluna de pesos (classe 1 com peso maior)
train_balanced = train_balanced.withColumn(
    "classWeightCol", when(col("label") == 1, 5.0).otherwise(1.0)
)

# GBT com weightCol
gbt = GBTClassifier(
    labelCol="label",
    featuresCol="features_topK",
    weightCol="classWeightCol",
    maxIter=20,
    maxDepth=5,
    seed=42
)

# Treinar modelo
model = gbt.fit(train_balanced)

# Inferência
val_preds = model.transform(val_topk)

# Avaliação
f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(val_preds)
precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision").evaluate(val_preds)
recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall").evaluate(val_preds)

print(f"\n✅ GBT com Pesos:")
print(f"F1-score:  {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")

# Matriz de confusão
preds_rdd = val_preds.select("prediction", "label").rdd.map(lambda r: (float(r[0]), float(r[1])))
metrics = MulticlassMetrics(preds_rdd)
print("\nConfusion Matrix:")
print(metrics.confusionMatrix().toArray())

# Guardar modelo
model.write().overwrite().save("/FileStore/models/gbt_top10_weighted_model")


✅ GBT com Pesos:
F1-score:  0.3053
Precision: 0.9655
Recall:    0.2152

Confusion Matrix:
[[1.32724e+05 5.81197e+05]
 [4.00000e+00 2.66940e+04]]
